In [ ]:
import pandas as pd
import sqlalchemy as db
import psycopg2
import os
from sqlalchemy import create_engine
import os, time
import csv
import sqlalchemy
import win32com.client as win32
from datetime import datetime
from datetime import timedelta
from datetime import date
from zipfile import ZipFile
import glob, os
import shutil

In [ ]:
#place the file to be uploaded in C:/Power BI/single_upload/
parent_dir = 'C:/Power BI/single_upload/' #path to folder
path = parent_dir

searchdir = parent_dir

for xls_file in glob.glob(os.path.join(searchdir,"*.xlsx")):
    data_xls = pd.read_excel(xls_file, index_col = None)
    csv_file = os.path.splitext(xls_file)[0]+".csv"
    data_xls.to_csv(csv_file, encoding = 'utf-8', index = False)
    
print('File Changed to CSV')

#rename files longer than >=63 char
for filename in os.listdir(path):
    if len(filename) > 63:
        os.rename(path+filename, path+filename[-60:])
        print(filename+' renamed to '+filename[-60:])
            
#get csv list
csv_files = []
for file in os.listdir(path):
    if file.endswith('.csv'):
        csv_files.append(file)
        
data_path = path
#create dataframes
df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file, low_memory=False, index_col=False)
        
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="cp437", low_memory=False, index_col=False, errors='ignore')
    
    print('Loading ' + file + ' into dataframe')    
print('loading completed')

In [ ]:
for k in csv_files:
    
    dataframe = df[k]
    
    clean_tbl_name = k.lower().replace(" ","_").replace("-","").replace(".", "_").replace("(", "").replace(")", "").replace(",", "")\
    .replace("_csv", "").replace("___", "_").replace("__", "_")    
    
    tbl_name = clean_tbl_name
    
    print(k + ' changing to ' + clean_tbl_name)

    #clean column names
    dataframe.columns = [x.lower().replace(" ", "_").replace("-", "").replace("#","num").replace("?", "")\
                     .replace("=","").replace("\n","").replace("\r","").replace("\r\n","").replace("]","_")\
                     .replace("]","_").replace("[","_").replace("\\","_").replace(".","_").replace("$","")\
                     .replace("%","").replace("#","").replace("(","").replace(")","").replace("?","")\
                     .replace(",","").replace("*","").replace(":","").replace("'","").replace("&","")\
                     .replace(";","").replace("__", "_").replace("/", "")
                     for x in dataframe.columns]

     #limit column length to 64 and reading right to left
    dataframe.columns = dataframe.columns.str[-60:] 

     #adding a number if duplicated column name
    def uniquify(dataframe):
        seen = set()

        for item in dataframe:
            fudge = 1
            newitem = item

            while newitem in seen:
                fudge += 1
                newitem = "{}_{}".format(item, fudge)

            yield newitem
            seen.add(newitem)

    dataframe.columns = uniquify(dataframe)

    dataframe.columns = dataframe.columns.str[-60:] 
    
    #db settings and connection
    #get password
    f=open("S:/Data Team Secure/secrets/postgres.txt","r")
    lines=f.readlines()
    password=lines[1]
    f.close()
    
    user="CASADataTeam"
    host = 'casa-dev.postgres.database.azure.com'
    dbname = 'postgres'
        
    engine = create_engine('postgresql://'+user+':'+password+'@'+host+'/'+dbname)
   
    #print('opened database successfully')
    
    #create table
    #dataframe.to_sql(k, engine, schema = None, if_exists='append', index=False, dtype = 'text')
    dataframe.to_sql(clean_tbl_name, engine, schema = None, if_exists='append', index=False, dtype =  
                             {'datefld': sqlalchemy.DateTime(), 
                             'intfld':  sqlalchemy.types.INTEGER(),
                             'strfld': sqlalchemy.types.NVARCHAR(length=255),
                             'floatfld': sqlalchemy.types.Float(precision=3, asdecimal=True),
                             'booleanfld': sqlalchemy.types.Boolean,
                             'bool' : sqlalchemy.types.Boolean,
                             'float64' : sqlalchemy.types.NVARCHAR(length=255),
                             'int64' : sqlalchemy.types.INTEGER(),
                             'object' : sqlalchemy.types.NVARCHAR(length=50000)})
    
    print(clean_tbl_name+' uploaded to database')
    
print('All uploads complete')